# Database exploration with robotoff API

In [4]:
%load_ext autoreload
%autoreload 2

In [4]:
from robotoff.products import ProductDataset

#streaming data
ds = ProductDataset.load()

product_iter = (ds.stream()
                  #.filter_by_country_tag('en:france')
                  #.filter_nonempty_text_field('ingredients_text_fr')
                  #.filter_by_state_tag('en:complete')
                  .iter())

count = 0
for product in product_iter:
    print(product['product_name'])
    count += 1
    if count == 10:
        break


Chocolat au lait
Vitória crackers
 moutarde au moût de raisin 
Sauce Sweety chili 0%
Mendiants
Salade de carottes râpées
Fromage blanc aux myrtilles
Solène céréales poulet
Tarte noix de coco


In [5]:
len(list(product_iter))

KeyboardInterrupt: 

In [19]:
prod = next(product_iter)
prod

{'_id': '0034500151924',
 'ingredients': [{'percent_max': 100.0,
   'percent_min': 50.0,
   'id': 'en:Sweet cream',
   'percent_estimate': 75.0,
   'text': 'Sweet cream'},
  {'vegetarian': 'maybe',
   'id': 'en:natural-flavouring',
   'percent_estimate': 25,
   'text': 'natural flavoring',
   'percent_max': 50.0,
   'vegan': 'maybe',
   'percent_min': 0.0}],
 'packaging_old': 'Butter sticks wrapped in paper in a light cardboard box',
 'update_key': 'ingredients20220627',
 'categories_properties_tags': ['all-products',
  'categories-known',
  'agribalyse-food-code-unknown',
  'agribalyse-proxy-food-code-16400',
  'agribalyse-proxy-food-code-known',
  'ciqual-food-code-16414',
  'ciqual-food-code-known',
  'agribalyse-known',
  'agribalyse-16400'],
 'other_nutritional_substances_tags': [],
 'interface_version_modified': '20150316.jqm2',
 'food_groups': 'en:fats',
 'ingredients_original_tags': ['en:Sweet cream', 'en:natural-flavouring'],
 'origins_tags': ['en:united-states'],
 'nutriments

## Collecting images from mongoDB

-> https://wiki.openfoodfacts.org/Developer-How_To

We need to convert each barcode (_id) to something of this form: https://images.openfoodfacts.org/images/products/073/762/806/4502/

The problem is that it becomes a guessing game which type of images exist: 

https://world.openfoodfacts.org/images/products/003/450/015/1924/front_en.5.400.jpg

In [20]:
from EDA import get_images
get_images(prod)

In [38]:
barcode = prod["_id"]

base = "https://images.openfoodfacts.org/images/products/"

def to_pattern(s):
    pat = []
    group_enumerator = list(range(0, len(s), 3))
    for index, i in enumerate(group_enumerator):
        if index == len(group_enumerator)-1: #rest
            if len(s[i:]) > 0:
                pat[-1] = pat[-1] + s[i:]
                continue
        pat.append(s[i:i+3])
    #[s[i:i+3] for i in range(0, len(s), 3)]
    return "/".join(pat)

to_url = lambda id: base+to_pattern(id)
barcode, to_pattern(barcode), to_url(barcode)
#https://images.openfoodfacts.org/images/products/
#https://world.openfoodfacts.org/images/products/003/450/015/1924/front_en.5.400.jpg

('0034500151924',
 '003/450/015/1924',
 'https://images.openfoodfacts.org/images/products/003/450/015/1924')

In [ ]:
# translate images map to 

## Collecting some sample question-answers

## Alternative: CSV exploration
Already contains the URLs to the images for which to make PDF files

In [2]:
import os
import pandas as pd 

In [ ]:
path = os.path.expanduser("~/code/DUchallenge/data/OFF/robotoff/datasets/en.openfoodfacts.org.products.csv") #english
df = pd.read_csv(path, sep="\t") 
# 937 415 produits for French version [is it a subset?]

In [ ]:
# 2.5M records ~2,538,819 products
#dict(df.iloc[1])
#len(df)
#df.head()
df.head()

In [ ]:
import json
import numpy as np 
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)
    
print(json.dumps(dict(df.iloc[1]), indent=4, cls=NumpyEncoder))

In [ ]:
#print(df.columns.tolist())
most_important = ["product_name"]
original_columns = ['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'product_name', 'abbreviated_product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'packaging_en', 'packaging_text', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_en', 'origins', 'origins_tags', 'origins_en', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_en', 'ingredients_text', 'ingredients_tags', 'allergens', 'allergens_en', 'traces', 'traces_tags', 'traces_en', 'serving_size', 'serving_quantity', 'no_nutriments', 'additives_n', 'additives', 'additives_tags', 'additives_en', 'nutriscore_score', 'nutriscore_grade', 'nova_group', 'pnns_groups_1', 'pnns_groups_2', 'food_groups', 'food_groups_tags', 'food_groups_en', 'states', 'states_tags', 'states_en', 'brand_owner', 'ecoscore_score', 'ecoscore_grade', 'main_category', 'main_category_en', 'image_url', 'image_small_url', 'image_ingredients_url', 'image_ingredients_small_url', 'image_nutrition_url', 'image_nutrition_small_url', 'energy-kj_100g', 'energy-kcal_100g', 'energy_100g', 'energy-from-fat_100g', 'fat_100g', 'saturated-fat_100g', '-butyric-acid_100g', '-caproic-acid_100g', '-caprylic-acid_100g', '-capric-acid_100g', '-lauric-acid_100g', '-myristic-acid_100g', '-palmitic-acid_100g', '-stearic-acid_100g', '-arachidic-acid_100g', '-behenic-acid_100g', '-lignoceric-acid_100g', '-cerotic-acid_100g', '-montanic-acid_100g', '-melissic-acid_100g', 'monounsaturated-fat_100g', 'polyunsaturated-fat_100g', 'omega-3-fat_100g', '-alpha-linolenic-acid_100g', '-eicosapentaenoic-acid_100g', '-docosahexaenoic-acid_100g', 'omega-6-fat_100g', '-linoleic-acid_100g', '-arachidonic-acid_100g', '-gamma-linolenic-acid_100g', '-dihomo-gamma-linolenic-acid_100g', 'omega-9-fat_100g', '-oleic-acid_100g', '-elaidic-acid_100g', '-gondoic-acid_100g', '-mead-acid_100g', '-erucic-acid_100g', '-nervonic-acid_100g', 'trans-fat_100g', 'cholesterol_100g', 'carbohydrates_100g', 'sugars_100g', '-sucrose_100g', '-glucose_100g', '-fructose_100g', '-lactose_100g', '-maltose_100g', '-maltodextrins_100g', 'starch_100g', 'polyols_100g', 'fiber_100g', 'soluble-fiber_100g', 'insoluble-fiber_100g', 'proteins_100g', 'casein_100g', 'serum-proteins_100g', 'nucleotides_100g', 'salt_100g', 'sodium_100g', 'alcohol_100g', 'vitamin-a_100g', 'beta-carotene_100g', 'vitamin-d_100g', 'vitamin-e_100g', 'vitamin-k_100g', 'vitamin-c_100g', 'vitamin-b1_100g', 'vitamin-b2_100g', 'vitamin-pp_100g', 'vitamin-b6_100g', 'vitamin-b9_100g', 'folates_100g', 'vitamin-b12_100g', 'biotin_100g', 'pantothenic-acid_100g', 'silica_100g', 'bicarbonate_100g', 'potassium_100g', 'chloride_100g', 'calcium_100g', 'phosphorus_100g', 'iron_100g', 'magnesium_100g', 'zinc_100g', 'copper_100g', 'manganese_100g', 'fluoride_100g', 'selenium_100g', 'chromium_100g', 'molybdenum_100g', 'iodine_100g', 'caffeine_100g', 'taurine_100g', 'ph_100g', 'fruits-vegetables-nuts_100g', 'fruits-vegetables-nuts-dried_100g', 'fruits-vegetables-nuts-estimate_100g', 'fruits-vegetables-nuts-estimate-from-ingredients_100g', 'collagen-meat-protein-ratio_100g', 'cocoa_100g', 'chlorophyl_100g', 'carbon-footprint_100g', 'carbon-footprint-from-meat-or-fish_100g', 'nutrition-score-fr_100g', 'nutrition-score-uk_100g', 'glycemic-index_100g', 'water-hardness_100g', 'choline_100g', 'phylloquinone_100g', 'beta-glucan_100g', 'inositol_100g', 'carnitine_100g']
image_columns = [x for x in original_columns if "image" in x]
image_columns

## Statistics on the dataset [before cleaning]

1. kick out variables that are not filled in 25% of cases

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def null_factor(df, tx_threshold=50):
    null_rate = ((df.isnull().sum() / df.shape[0])*100).sort_values(ascending=False).reset_index()
    null_rate.columns = ['Variable','Taux_de_Null']
    high_null_rate = null_rate[null_rate.Taux_de_Null >= tx_threshold]
    return high_null_rate

full_null_rate = null_factor(df, 100)

filling_features = null_factor(df, 0)
filling_features["Taux_de_Null"] = 100-filling_features["Taux_de_Null"]
filling_features = filling_features.sort_values("Taux_de_Null", ascending=False) 

#Seuil de suppression
sup_threshold = 25


#Liste des variables à conserver
features_to_conserve = list(filling_features.loc[filling_features['Taux_de_Null']>=sup_threshold, 'Variable'].values)
#Liste des variables supprimées
deleted_features = list(filling_features.loc[filling_features['Taux_de_Null']<sup_threshold, 'Variable'].values)

#Nouveau dfet avec les variables conservées
df = df[features_to_conserve].sort_values(["created_datetime","last_modified_datetime"], ascending=True)
print(features_to_conserve)

plot = True

if plot:

    fig = plt.figure(figsize=(20, 35))

    font_title = {'family': 'serif',
                  'color':  '#114b98',
                  'weight': 'bold',
                  'size': 18,
                 }


    sns.barplot(x="Taux_de_Null", y="Variable", data=filling_features, palette="flare")
    #Seuil pour suppression des varaibles
    plt.axvline(x=sup_threshold, linewidth=2, color = 'r')
    plt.text(sup_threshold+2, 65, 'Seuil de suppression des variables', fontsize = 16, color = 'r')

    plt.title("Taux de remplissage des variables dans le jeu de données (%)", fontdict=font_title)
    plt.xlabel("Taux de remplissage (%)")
    plt.show()

In [ ]:
# filter for where all images are present 
sorted(features_to_conserve)
image_columns = [
 'image_ingredients_small_url',
 'image_ingredients_url',
 'image_nutrition_small_url',
 'image_nutrition_url',
 'image_small_url',
 'image_url']
#it seems front and back are missing? -> no distinction in csv

image_df = df[(~df.image_url.isnull()) & (~df.image_ingredients_url.isnull()) & (~df.image_nutrition_url.isnull())]
print(len(image_df))
image_df.columns

#434617

#lambda get images

In [ ]:
#image_df
from EDA import get_images
#stop 1: 963
#stop 2 5392
#stop 3: 148124

from tqdm import tqdm
count = 0
for i, row in tqdm(image_df.iterrows()):
    if count < 148124: #stopped at 963
        count += 1
        continue
    try:
        get_images(dict(row), save=True, identifier=str(row["code"]), binary=True)
    except Exception as e:
        print(f"{count}: {i}, {e}")
    count += 1
    #optional OCR

In [ ]:
4876it [00:01, 5591.94it/s]
5392: 980122, cannot identify image file <_io.BytesIO object at 0x7f6045165220>
7916it [29:26,  1.53it/s]  
7915: 1631032, cannot identify image file <_io.BytesIO object at 0x7f6045165bd0>
9237it [44:53,  4.52s/it]
9236: 1457942, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/404/488/900/1006/front_de.58.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60451161f0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
9666it [49:49,  1.78it/s]
9665: 1203452, cannot identify image file <_io.BytesIO object at 0x7f604511ddb0>
19070it [2:33:34,  7.06s/it]
19069: 948154, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/309/575/106/7010/front_fr.748.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f6098f5e280>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
19664it [2:40:21,  5.64s/it]
19663: 967307, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/317/466/009/7544/ingredients_fr.22.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f5f053b2ee0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
20318it [2:47:50,  2.12it/s]
20317: 1787648, cannot identify image file <_io.BytesIO object at 0x7f604517ba40>
28766it [5:04:58,  1.26it/s]
28765: 1026607, cannot identify image file <_io.BytesIO object at 0x7f6098f5e540>
30042it [5:25:45,  6.77s/it]
30041: 1038309, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/326/385/128/3610/ingredients_fr.11.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f6098f491f0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
32265it [5:49:50,  2.09it/s]
32264: 1309223, cannot identify image file <_io.BytesIO object at 0x7f604517ba90>
35164it [6:20:57,  1.60it/s]
35163: 1397108, cannot identify image file <_io.BytesIO object at 0x7f6098f4d4a0>
36022it [6:30:30,  8.01s/it]
36021: 2399185, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/871/360/018/6693/nutrition_fr.7.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f6098f49280>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
36031it [6:31:00,  8.32s/it]
36030: 138907, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/004/134/510/5128/ingredients_en.9.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f6098f49f70>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
36032it [6:31:21, 11.91s/it]
36031: 150216, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/004/150/000/2248/ingredients_en.8.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f6098f49670>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
42377it [7:39:02,  1.70it/s]
42376: 940132, cannot identify image file <_io.BytesIO object at 0x7f6045203e50>
42812it [7:45:25, 19.05s/it]
42811: 1519464, cannot identify image file <_io.BytesIO object at 0x7f6045203ea0>
42816it [7:46:27, 22.89s/it]
42815: 1204671, cannot identify image file <_io.BytesIO object at 0x7f6045203680>
42831it [7:47:37, 18.60s/it]
42830: 961281, cannot identify image file <_io.BytesIO object at 0x7f604520c770>
42882it [7:49:09, 18.53s/it]
42881: 1641194, cannot identify image file <_io.BytesIO object at 0x7f6098f4dcc0>
43034it [7:52:12, 18.67s/it]
43033: 1207639, cannot identify image file <_io.BytesIO object at 0x7f6045203770>
43110it [7:53:59, 18.47s/it]
43109: 1458369, cannot identify image file <_io.BytesIO object at 0x7f6045203040>
43112it [7:55:00, 27.24s/it]
43111: 1439134, cannot identify image file <_io.BytesIO object at 0x7f6098f42ae0>
43152it [7:56:24, 18.65s/it]
43151: 719943, cannot identify image file <_io.BytesIO object at 0x7f6045203680>
43231it [7:58:13, 18.50s/it]
43230: 747398, cannot identify image file <_io.BytesIO object at 0x7f6045203040>
43285it [8:00:06, 20.45s/it]
43284: 736219, cannot identify image file <_io.BytesIO object at 0x7f6098f42810>
43292it [8:01:10, 20.10s/it]
43291: 1082947, cannot identify image file <_io.BytesIO object at 0x7f60452036d0>
43354it [8:03:28, 30.59s/it]
43353: 2193593, cannot identify image file <_io.BytesIO object at 0x7f6045203040>
43530it [8:06:56, 18.57s/it]
43529: 720231, cannot identify image file <_io.BytesIO object at 0x7f6045203770>
43619it [8:10:59, 18.56s/it]
43618: 1965339, cannot identify image file <_io.BytesIO object at 0x7f60452036d0>
43626it [8:12:04, 20.04s/it]
43625: 924722, cannot identify image file <_io.BytesIO object at 0x7f604520c770>
43729it [8:14:09, 18.60s/it]
43728: 1071040, cannot identify image file <_io.BytesIO object at 0x7f6045203040>
43793it [8:15:49, 18.55s/it]
43792: 872457, cannot identify image file <_io.BytesIO object at 0x7f6045203770>
43875it [8:17:38, 18.51s/it]
43874: 1549274, cannot identify image file <_io.BytesIO object at 0x7f6045203360>
43929it [8:19:41, 19.23s/it]
43928: 1197050, cannot identify image file <_io.BytesIO object at 0x7f6045203770>
44001it [8:21:28, 18.59s/it]
44000: 873007, cannot identify image file <_io.BytesIO object at 0x7f60452036d0>
44003it [8:22:29, 27.36s/it]
44002: 1421529, cannot identify image file <_io.BytesIO object at 0x7f6045203360>
44005it [8:23:30, 31.76s/it]
44004: 1002721, cannot identify image file <_io.BytesIO object at 0x7f60452036d0>
44024it [8:24:47, 18.57s/it]
44023: 720302, cannot identify image file <_io.BytesIO object at 0x7f6045203770>
51666it [9:44:49,  1.56it/s]
51665: 1395502, cannot identify image file <_io.BytesIO object at 0x7f6045237400>
53071it [9:59:29,  1.76it/s]
53070: 936270, cannot identify image file <_io.BytesIO object at 0x7f60452371d0>
55990it [10:30:10,  1.98it/s]
55989: 1419869, cannot identify image file <_io.BytesIO object at 0x7f6045237180>
56319it [10:33:35,  1.83it/s]
56318: 1142454, cannot identify image file <_io.BytesIO object at 0x7f60452424f0>
57632it [10:47:10,  2.11it/s]
57631: 1668388, cannot identify image file <_io.BytesIO object at 0x7f6045237090>
59115it [11:02:52,  1.87it/s]
59114: 2441006, cannot identify image file <_io.BytesIO object at 0x7f60451fa7c0>
59205it [11:03:47,  1.61it/s]
59204: 981600, cannot identify image file <_io.BytesIO object at 0x7f604524e2c0>
59804it [11:10:05,  2.10it/s]
59803: 1017536, cannot identify image file <_io.BytesIO object at 0x7f604524ecc0>
60042it [11:12:21,  1.12it/s]
60041: 732380, cannot identify image file <_io.BytesIO object at 0x7f604524ef40>
60385it [11:15:30,  1.79it/s]
60384: 1582480, cannot identify image file <_io.BytesIO object at 0x7f6045237090>
60450it [11:16:05,  2.12it/s]
60449: 1206140, cannot identify image file <_io.BytesIO object at 0x7f604524e4a0>
60474it [11:16:18,  1.83it/s]
60473: 1665603, cannot identify image file <_io.BytesIO object at 0x7f604524e270>
60930it [11:20:39,  1.93it/s]
60929: 1930578, cannot identify image file <_io.BytesIO object at 0x7f604524e860>
62467it [11:35:36,  2.01it/s]
62466: 983517, cannot identify image file <_io.BytesIO object at 0x7f60452422c0>
63686it [11:47:42,  6.07s/it]
63685: 938848, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/305/723/000/0253/front_fr.30.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60451b2280>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
64806it [11:58:40,  6.45s/it]
64805: 1019794, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/325/622/516/4240/ingredients_fr.43.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60451b2280>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
68826it [12:37:30,  2.09it/s]
68825: 1005066, cannot identify image file <_io.BytesIO object at 0x7f60451af130>
73115it [13:18:47,  1.84it/s]
73114: 951082, cannot identify image file <_io.BytesIO object at 0x7f60451c61d0>
74719it [13:34:13,  1.66it/s]
74718: 924495, cannot identify image file <_io.BytesIO object at 0x7f60451cfbd0>
75269it [13:39:37,  1.26s/it]
75268: 983578, cannot identify image file <_io.BytesIO object at 0x7f60451cf770>
80863it [14:35:35,  1.52it/s]
80862: 1757503, cannot identify image file <_io.BytesIO object at 0x7f60451cfae0>
81846it [14:45:42,  2.06it/s]
81845: 1044450, cannot identify image file <_io.BytesIO object at 0x7f60bcbc2950>
81893it [14:46:09,  2.28it/s]
81892: 947826, cannot identify image file <_io.BytesIO object at 0x7f60bcbc2db0>
85057it [15:17:28,  2.01it/s]
85056: 1238157, cannot identify image file <_io.BytesIO object at 0x7f60bcbd1180>
87501it [15:44:50,  8.04s/it]
87500: 1048658, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/326/619/106/0514/nutrition_fr.36.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60451cb9a0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
88106it [15:50:58,  2.14it/s]
88105: 1054452, cannot identify image file <_io.BytesIO object at 0x7f60bcbdb400>
88547it [15:55:33,  2.33it/s]
88546: 1121091, cannot identify image file <_io.BytesIO object at 0x7f60bcbdbd60>
90017it [16:11:23,  6.92s/it]
90016: 890265, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/27040805/front_fr.29.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bcbc8af0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
90497it [16:17:15,  2.29it/s]
90496: 1121067, cannot identify image file <_io.BytesIO object at 0x7f60bcbe89f0>
90899it [16:21:20,  1.37it/s]
90898: 988270, cannot identify image file <_io.BytesIO object at 0x7f60bcbe88b0>
91552it [16:28:08,  1.84it/s]
91551: 1030517, cannot identify image file <_io.BytesIO object at 0x7f60bcbe8db0>
92052it [16:33:09,  2.01it/s]
92051: 1249011, cannot identify image file <_io.BytesIO object at 0x7f60bcbe8db0>
92178it [16:34:23,  1.89it/s]
92177: 756848, cannot identify image file <_io.BytesIO object at 0x7f60bcbc2950>
92887it [16:41:34,  1.42s/it]
92886: 983771, cannot identify image file <_io.BytesIO object at 0x7f60bcbf8b80>
92952it [16:42:19,  1.90it/s]
92951: 1166026, cannot identify image file <_io.BytesIO object at 0x7f60bcbf8e50>
95833it [17:11:41,  1.68it/s]
95832: 1063109, cannot identify image file <_io.BytesIO object at 0x7f60bc9cbe00>
96151it [17:14:39,  1.76it/s]
96150: 1145262, cannot identify image file <_io.BytesIO object at 0x7f60bc9cbdb0>
97385it [17:26:22,  1.82it/s]
97384: 796838, cannot identify image file <_io.BytesIO object at 0x7f60bc9cbcc0>
98802it [17:40:18,  1.55it/s]
98801: 1071547, cannot identify image file <_io.BytesIO object at 0x7f60bc9d1950>
99445it [17:46:52,  6.45s/it]
99444: 1924726, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/762/220/000/6793/ingredients_fr.7.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bc9c9700>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
99954it [17:51:49,  1.71it/s]
99953: 1214918, cannot identify image file <_io.BytesIO object at 0x7f60bc9d1810>
101262it [18:04:05,  1.76it/s]
101261: 458750, cannot identify image file <_io.BytesIO object at 0x7f60bc9e9db0>
101453it [18:06:17,  2.19it/s]
101452: 1527176, cannot identify image file <_io.BytesIO object at 0x7f60bc9e9e00>
102089it [18:12:47,  6.66s/it]
102088: 1073011, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/328/274/038/0247/nutrition_fr.27.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bc9f51f0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
102090it [18:13:07, 10.67s/it]
102089: 1218938, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/356/470/079/8744/front_fr.4.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bc9f5490>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
102091it [18:13:27, 13.48s/it]
102090: 752654, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/20711825/front_fr.14.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bc9f5460>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
102100it [18:13:57,  7.03s/it]
102099: 2251595, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/842/462/100/0393/ingredients_fr.4.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bc9c9df0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
102101it [18:14:17, 10.93s/it]
102100: 1239817, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/359/671/041/3850/front_fr.8.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bcbea190>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
102102it [18:14:27, 10.67s/it]
102101: 1127943, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/338/038/007/4356/front_fr.5.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bcbeaac0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
102109it [18:14:51,  7.30s/it]
102108: 1179273, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/349/897/619/9984/ingredients_fr.4.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bc9c93d0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
102110it [18:15:01,  8.12s/it]
102109: 1240302, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/359/671/042/4368/front_fr.30.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bc9c96d0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
102387it [18:17:37,  2.35it/s]
102386: 984179, cannot identify image file <_io.BytesIO object at 0x7f60bc9e9a40>
102412it [18:18:11,  6.52s/it]
102411: 1998526, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/800/247/002/3864/ingredients_fr.7.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bc9f5430>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
103697it [18:30:32,  2.15it/s]
103696: 985043, cannot identify image file <_io.BytesIO object at 0x7f60bc9f3ae0>
104891it [18:42:03,  2.14it/s]
104890: 1014471, cannot identify image file <_io.BytesIO object at 0x7f60bc9e94a0>
106724it [18:59:44,  1.61it/s]
106723: 976035, cannot identify image file <_io.BytesIO object at 0x7f60bc6bff90>
108398it [19:14:42,  2.16it/s]
108397: 1046672, cannot identify image file <_io.BytesIO object at 0x7f60bc6bfa90>
109176it [19:21:34,  2.15it/s]
109175: 1217957, cannot identify image file <_io.BytesIO object at 0x7f60bc6cd3b0>
110097it [19:29:42,  2.24it/s]
110096: 747032, cannot identify image file <_io.BytesIO object at 0x7f60bc6bf130>
110122it [19:29:55,  2.48it/s]
110121: 363241, cannot identify image file <_io.BytesIO object at 0x7f60bc6cd310>
112198it [19:48:34,  2.46it/s]
112197: 1015146, cannot identify image file <_io.BytesIO object at 0x7f60bc6cd310>
112232it [19:48:53,  2.15it/s]
112231: 981850, cannot identify image file <_io.BytesIO object at 0x7f60bc6dfef0>
112375it [19:50:10,  2.33it/s]
112374: 984796, cannot identify image file <_io.BytesIO object at 0x7f60bc6cd040>
112392it [19:50:44,  7.92s/it]
112391: 1156169, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/343/949/900/2269/ingredients_fr.7.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bc6d7be0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
112393it [19:50:59, 10.05s/it]
112392: 1103206, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/333/704/000/0064/front_fr.8.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bc6d78e0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
112401it [19:51:13,  4.01s/it]
112400: 756785, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/20841461/nutrition_fr.14.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bc9f2490>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
113348it [19:59:43,  2.12it/s]
113347: 1181706, cannot identify image file <_io.BytesIO object at 0x7f60bc6dfbd0>
115346it [20:19:49,  5.91s/it]
115345: 738510, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/20289638/front_fr.22.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bc6d7130>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
115462it [20:21:05,  5.00s/it]
115461: 722567, HTTPSConnectionPool(host='images.openfoodfacts.org', port=443): Max retries exceeded with url: /images/products/20039424/nutrition_fr.14.400.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f60bc9f29d0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
116206it [20:27:42,  2.02it/s]
116205: 1175080, cannot identify image file <_io.BytesIO object at 0x7f60bc6df900>
116876it [20:33:40,  1.80it/s]
116875: 1793851, cannot identify image file <_io.BytesIO object at 0x7f60bc6eb540>
118012it [20:44:37,  1.77it/s]
118011: 987951, cannot identify image file <_io.BytesIO object at 0x7f60bc6eb5e0>
120376it [21:06:08,  7.00s/it]
120375: 1418015, cannot identify image file <_io.BytesIO object at 0x7f60bc6cd040>
120385it [21:06:15,  1.32s/it]
120384: 1675985, cannot identify image file <_io.BytesIO object at 0x7f60bc6eba90>
121139it [21:13:03,  2.01it/s]
121138: 1131145, cannot identify image file <_io.BytesIO object at 0x7f60bc6df900>
121952it [21:20:13,  1.87it/s]
121951: 1232041, cannot identify image file <_io.BytesIO object at 0x7f60bc6ed040>
122208it [21:22:57,  1.86it/s]
122207: 1071845, cannot identify image file <_io.BytesIO object at 0x7f60bc6edae0>
123290it [21:32:40,  2.18it/s]
123289: 1006588, cannot identify image file <_io.BytesIO object at 0x7f60bbc81a90>
126724it [22:03:30,  1.51it/s]
126723: 1118878, cannot identify image file <_io.BytesIO object at 0x7f60bbc811d0>
127197it [22:07:42,  2.11it/s]
127196: 1074099, cannot identify image file <_io.BytesIO object at 0x7f60bc6ed090>
128876it [22:22:28,  2.19it/s]
128875: 928050, cannot identify image file <_io.BytesIO object at 0x7f60bbc8f4a0>
130089it [22:33:03,  1.91it/s]
130088: 1823077, cannot identify image file <_io.BytesIO object at 0x7f60bbc99720>
130265it [22:34:35,  1.92it/s]
130264: 1240549, cannot identify image file <_io.BytesIO object at 0x7f60bbc99040>
134215it [23:09:32,  2.10it/s]
134214: 1330265, cannot identify image file <_io.BytesIO object at 0x7f60bbc99220>
134575it [23:12:43,  2.22it/s]
134574: 1006931, cannot identify image file <_io.BytesIO object at 0x7f60bbcaad60>
135148it [23:17:47,  1.87it/s]
135147: 988624, cannot identify image file <_io.BytesIO object at 0x7f60bbcaa9f0>
135671it [23:22:28,  2.09it/s]
135670: 1257431, cannot identify image file <_io.BytesIO object at 0x7f60bbcaa9f0>
136756it [23:32:00,  2.27it/s]
136755: 225368, cannot identify image file <_io.BytesIO object at 0x7f60bbcaa0e0>
137281it [23:36:35,  2.02it/s]
137280: 1207634, cannot identify image file <_io.BytesIO object at 0x7f60bbcbbea0>
137753it [23:41:03,  1.11s/it]
137752: 1385547, cannot identify image file <_io.BytesIO object at 0x7f60bbc99220>
137789it [23:41:23,  2.09it/s]
137788: 756764, cannot identify image file <_io.BytesIO object at 0x7f60bbcbba90>
139136it [23:53:19,  2.16it/s]
139135: 1519793, cannot identify image file <_io.BytesIO object at 0x7f60bbcbbf90>
139326it [23:54:58,  2.03it/s]
139325: 1220065, cannot identify image file <_io.BytesIO object at 0x7f60bbcbb8b0>
139332it [23:55:00,  2.63it/s]
139331: 1280312, cannot identify image file <_io.BytesIO object at 0x7f60bbcbbf90>
139878it [23:59:52,  2.34it/s]
139877: 1057337, cannot identify image file <_io.BytesIO object at 0x7f60bbcbba90>
140179it [24:02:30,  2.23it/s]
140178: 990242, cannot identify image file <_io.BytesIO object at 0x7f60bbcbbef0>
140360it [24:04:04,  2.03it/s]
140359: 975283, cannot identify image file <_io.BytesIO object at 0x7f60b56c1540>
140690it [24:06:57,  2.08it/s]
140689: 1005991, cannot identify image file <_io.BytesIO object at 0x7f60bbcbba90>
140788it [24:07:49,  2.10it/s]
140787: 150612, cannot identify image file <_io.BytesIO object at 0x7f60bbc99c70>
140826it [24:08:08,  2.29it/s]
140825: 2460623, cannot identify image file <_io.BytesIO object at 0x7f60bbcbb8b0>
140914it [24:08:56,  2.13it/s]
140913: 1001595, cannot identify image file <_io.BytesIO object at 0x7f60bbcbbef0>
141443it [24:13:36,  2.21it/s]
141442: 763877, cannot identify image file <_io.BytesIO object at 0x7f60bbc99220>
141468it [24:13:49,  2.17it/s]
141467: 1783525, cannot identify image file <_io.BytesIO object at 0x7f60bbcaa0e0>
141721it [24:15:59,  2.23it/s]
141720: 1052852, cannot identify image file <_io.BytesIO object at 0x7f60b56c1630>
141725it [24:16:01,  2.08it/s]
141724: 1321255, cannot identify image file <_io.BytesIO object at 0x7f60bbcaa040>
141819it [24:16:50,  2.15it/s]
141818: 1368983, cannot identify image file <_io.BytesIO object at 0x7f60b56c19f0>
142275it [24:20:46,  2.26it/s]
142274: 1242099, cannot identify image file <_io.BytesIO object at 0x7f60bbcbbf90>
142406it [24:21:53,  2.22it/s]
142405: 1003209, cannot identify image file <_io.BytesIO object at 0x7f60b56c1630>
142696it [24:24:22,  2.24it/s]
142695: 1270050, cannot identify image file <_io.BytesIO object at 0x7f60b56c19f0>
142738it [24:24:44,  1.48it/s]
142737: 1337757, cannot identify image file <_io.BytesIO object at 0x7f60b56c15e0>
142867it [24:25:52,  2.14it/s]
142866: 1127188, cannot identify image file <_io.BytesIO object at 0x7f60bbc99c70>
142965it [24:26:42,  2.01it/s]
142964: 1636540, cannot identify image file <_io.BytesIO object at 0x7f60b56c1f40>
143134it [24:28:10,  2.06it/s]
143133: 1119826, cannot identify image file <_io.BytesIO object at 0x7f60b56c19f0>
143200it [24:28:44,  1.85it/s]
143199: 2286186, cannot identify image file <_io.BytesIO object at 0x7f60bbcaa0e0>
143326it [24:29:50,  2.09it/s]
143325: 1093564, cannot identify image file <_io.BytesIO object at 0x7f60b56c1f40>
143393it [24:30:26,  2.03it/s]
143392: 1249110, cannot identify image file <_io.BytesIO object at 0x7f60b56c12c0>
143661it [24:32:46,  2.19it/s]
143660: 784923, cannot identify image file <_io.BytesIO object at 0x7f60b56c19f0>
143681it [24:32:56,  2.15it/s]
143680: 1226439, cannot identify image file <_io.BytesIO object at 0x7f60bbcbbf90>
143797it [24:33:57,  2.09it/s]
143796: 1109982, cannot identify image file <_io.BytesIO object at 0x7f60bbcbbef0>
143816it [24:34:07,  2.03it/s]
143815: 1044398, cannot identify image file <_io.BytesIO object at 0x7f60bbcbbf90>
143884it [24:34:45,  1.94it/s]
143883: 967085, cannot identify image file <_io.BytesIO object at 0x7f60bbcaa040>
143905it [24:34:56,  2.13it/s]
143904: 1226435, cannot identify image file <_io.BytesIO object at 0x7f60bbcbbef0>
143938it [24:35:14,  1.79it/s]
143937: 929060, cannot identify image file <_io.BytesIO object at 0x7f60bbcaa040>
144156it [24:37:09,  1.92it/s]
144155: 952229, cannot identify image file <_io.BytesIO object at 0x7f60bbcbbef0>
144161it [24:37:12,  1.96it/s]
144160: 1270234, cannot identify image file <_io.BytesIO object at 0x7f60bbcbbef0>
144191it [24:37:28,  2.14it/s]
144190: 1033798, cannot identify image file <_io.BytesIO object at 0x7f60b56c1a90>
144313it [24:38:34,  2.17it/s]
144312: 1103228, cannot identify image file <_io.BytesIO object at 0x7f60bbc99220>
144597it [24:41:10,  1.95it/s]
144596: 1069367, cannot identify image file <_io.BytesIO object at 0x7f60b56cf900>
144626it [24:41:25,  2.11it/s]
144625: 1138716, cannot identify image file <_io.BytesIO object at 0x7f60b56c1f40>
145058it [24:45:13,  2.17it/s]
145057: 1241218, cannot identify image file <_io.BytesIO object at 0x7f60b56cfae0>
145438it [24:48:30,  2.13it/s]
145437: 1749652, cannot identify image file <_io.BytesIO object at 0x7f60b56cff40>
145749it [24:51:12,  1.83it/s]
145748: 1846952, cannot identify image file <_io.BytesIO object at 0x7f60bbcbb8b0>
145962it [24:53:05,  2.10it/s]
145961: 44877, cannot identify image file <_io.BytesIO object at 0x7f60bbcbb8b0>
145964it [24:53:06,  2.14it/s]
145963: 1183333, cannot identify image file <_io.BytesIO object at 0x7f60bbcaa040>
146491it [24:57:53,  1.96it/s]
146490: 954098, cannot identify image file <_io.BytesIO object at 0x7f60b56cfcc0>
146518it [24:58:07,  1.93it/s]
146517: 886737, cannot identify image file <_io.BytesIO object at 0x7f60b56cf8b0>
146565it [24:58:32,  1.71it/s]
146564: 1112484, cannot identify image file <_io.BytesIO object at 0x7f60b56cfcc0>
146660it [24:59:29,  1.76it/s]
146659: 1279153, cannot identify image file <_io.BytesIO object at 0x7f60b56cfdb0>
146671it [24:59:35,  1.76it/s]
146670: 962720, cannot identify image file <_io.BytesIO object at 0x7f60b56cf4a0>
147242it [25:04:44,  1.75it/s]
147241: 1170240, cannot identify image file <_io.BytesIO object at 0x7f60bbcaa0e0>
147259it [25:04:53,  2.02it/s]
147258: 111394, cannot identify image file <_io.BytesIO object at 0x7f60b56cfd10>
147515it [25:08:02,  1.11s/it]
147514: 2202553, cannot identify image file <_io.BytesIO object at 0x7f60bbcbba90>
147574it [25:09:09,  1.09s/it]
147573: 1134422, cannot identify image file <_io.BytesIO object at 0x7f60b56cf950>
147791it [25:13:01,  1.10it/s]
147790: 1226605, cannot identify image file <_io.BytesIO object at 0x7f60b56cfd10>
148124it [25:18:43,  1.05s/it]

In [ ]:
import os
from fast_OCR import OCR_it
for i, row in tqdm(image_df.iterrows()):
    file = os.path.join("PDFs", str(row["code"])+".pdf")
    OCR_it(file)

In [32]:
image_df.countries.value_counts() #where sold -> language of packaging?

France                                          185194
España                                           21324
Deutschland                                      15398
en:Germany                                        8785
en:United States                                  6897
                                                 ...  
Таджикистан                                          1
France,Espagne,Royaume-Uni,en:united-kingdom         1
España,Francia,México,USA,Arabia Saudita             1
France,Portugal,Espagne,Suisse                       1
Denmark,Sweden,en:sweden                             1
Name: countries, Length: 8734, dtype: int64

In [29]:
#image_df.countries_tags.value_counts() #str.split(",")

image_df.countries_tags.apply(lambda x: str(x)[:2]).value_counts() #.value_counts() #str.split(",")


en    433096
na      1469
nl        19
fr        13
ro         7
pt         3
de         2
es         2
hu         1
ta         1
hr         1
ar         1
pl         1
it         1
Name: countries_tags, dtype: int64

In [76]:
image_df.to_csv("25drop-allimages-en-world.csv")
#countries_tags -> which country it was submitted

In [13]:
# check on languages
df["url"].apply(lambda x: x.split("-")[1][:2]).unique()
#http://world-en.openfoodfacts.org/product/0000...	


array(['en'], dtype=object)

In [66]:
# subset 100_g [phyicochemical substances]
def search_chem_component(df, suffix='_100g'):
    chem_component = []
    for col in df.columns:
        if '_100g' in col: chem_component.append(col)
    df_subset_columns = df[chem_component]
    return df_subset_columns

df_subset_nutrients = search_chem_component(df,'_100g')